In [ ]:
options(conflicts.policy = list(warn = FALSE))
library("dplyr")
library("estimatr")
library("lmtest")
library("plm")
library("readr")
library("rstatix")

In [ ]:
my_data <- read_csv("litigation.csv", show_col_types = 'FALSE')

In [ ]:
# egen id = group(country)
my_data <- my_data %>% group_by(country) %>% mutate(id = cur_group_id())

In [ ]:
# xtset id year
plm::is.pbalanced(my_data$id, my_data$year)

In [ ]:
my_data$lnghg <- log(my_data$ghg)
my_data$lngdp <- log(my_data$gdp)
my_data$lngdp2 <- log(my_data$gdp)^2
my_data$lnpopulation <- log(my_data$population)
my_data$lnlaw <- log(my_data$ruleoflaw)

In [ ]:
# sum lnghg lngdp lngdp2 lnpop lnlaw litigation
d <- data.frame(my_data$lnghg, my_data$lngdp, my_data$lngdp2, my_data$lnpopulation, my_data$lnlaw)
summary(d)

In [ ]:
# reg lnghg lngdp lngdp2 lnpopulation lnlaw litigation
mod <-lm(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + litigation, data = my_data)
summary(mod)

In [ ]:
#predict pred_lngdp
#predict pred_residuals, res
#scatter pred_residuals pred_lngdp
pred_lngdp <- predict(mod)
pred_residuals <- residuals(mod)
plot(y = pred_residuals, pred_lngdp)

In [ ]:
# hettest
lmtest::bptest(mod)

In [ ]:
# pwcorr lnpopulation lngdp, sig star(0.05)
cor.test(my_data$lnpopulation, my_data$lngdp)

In [ ]:
# pwcorr lnghg lngdp* lnpop litigation lnlaw, sig star(0.05)
my_data[,c("lnghg","lngdp", "lngdp2", "lnpopulation", "litigation", "lnlaw")] %>% cor_test(
  vars = c("lnghg"),
  vars2 = c("lngdp", "lngdp2","lnpopulation", "litigation", "lnlaw")
 )

In [ ]:
# pwcorr lnghg lngdp* lnpop litigation lnlaw, sig star(0.05)
my_data[,c("lnghg","lngdp", "lngdp2", "lnpopulation", "litigation", "lnlaw")] %>% cor_test()

In [ ]:
qqnorm(pred_residuals)

### TABLE 3 NEW

In [ ]:
# reg lnghg lngdp lngdp2 lnpopulation lnlaw litigation , robust
mod2 <-estimatr::lm_robust(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + litigation, data = my_data, se_type = "stata")
summary(mod2)

In [ ]:
# xtreg lnghg lngdp lngdp2 lnpopulation lnlaw litigation i.year, fe robust

modfe <- plm::plm(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + litigation,
                  index = c("id", "year"), model="within", effect='twoways', data = my_data)
summary(modfe, vcov=plm::vcovHC(modfe))

In [ ]:
HI <- my_data$country %in% c('BEL', 'AUS', 'CAN', 'CHE', 'DEU', 'ESP', 'EST', 'FIN', 
                             'FRA', 'GBR', 'GRC', 'HRV', 'ITA', 'LTU', 'LUX', 'NLD', 
                             'NOR', 'NZL', 'POL', 'PRT', 'SVK', 'SWE', 'ROU')
my_data$HITRUE_litigation <- HI * my_data$litigation
my_data$HIFALSE_litigation <- (1-HI) * my_data$litigation

In [ ]:
# reg lnghg lngdp lngdp2 lnpopulation lnlaw HI#c.litigation, robust

mod2 <-estimatr::lm_robust(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + HITRUE_litigation + HIFALSE_litigation, data = my_data, se_type = "stata")
summary(mod2)

In [ ]:
# xtreg lnghg lngdp lngdp2 lnpopulation lnlaw HI#c.litigation i.year, fe robust

modfe <- plm::plm(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + HITRUE_litigation + HIFALSE_litigation,
                  index = c("id", "year"), model="within", effect='twoways', data = my_data)
summary(modfe, vcov=plm::vcovHC(modfe))